<a href="https://colab.research.google.com/github/kaniarasann/Langchain/blob/main/07.Tools_with_Orchestration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
#!pip install langchain langgraph langchain_groq pydantic feedparser beautifulsoup4 langchain-community langchain-exa

Imports

In [26]:
from pydantic import BaseModel,Field
from datetime import datetime
from dateutil import parser as dtparser
import feedparser
from langchain_groq import ChatGroq
from google.colab import userdata
from langchain_core.prompts import PromptTemplate
from typing import List
import operator
from typing_extensions import Annotated
from langchain_exa import ExaSearchResults
from langgraph.constants import Send
from langgraph.graph import StateGraph, START, END
import re

Get top 5 news from the economic times this can be achieved with the help of RSS Feed : https://economictimes.indiatimes.com/markets/rssfeeds/1977021501.cms

In [27]:
url = 'https://economictimes.indiatimes.com/markets/rssfeeds/1977021501.cms'

In [28]:
# Class required for converting the Feed to pydantic class
class EconomicTime(BaseModel):
    title: str =''
    link: str = ''
    published: datetime =''
    content :str =''

In [29]:
#feed parse the Url add the nessecary package
feeds = feedparser.parse(url)

In [30]:
#iterate through the first 5 feed
economic_times:list[EconomicTime] = []
for feed in feeds.entries[:5]:
    economic_times.append(EconomicTime(
        title = feed.title,
        link = feed.link,
        published = dtparser.parse(feed.published)

    ))


In [31]:
economic_times

[EconomicTime(title='Is momentum only about price? ICICI Prudential wants to redefine momentum investing', link='https://economictimes.indiatimes.com/markets/expert-view/is-momentum-only-about-price-icici-prudential-wants-to-redefine-momentum-investing/articleshow/122796495.cms', published=datetime.datetime(2025, 7, 20, 16, 18, 42, tzinfo=tzoffset(None, 19800)), content=''),
 EconomicTime(title='Apollo Tyres, Brigade Enterprises among 10 small-cap stocks trading below industry PE; may rally up to 43%', link='https://economictimes.indiatimes.com/markets/stocks/news/apollo-tyres-brigade-enterprises-among-10-small-cap-stocks-trading-below-industry-pe-may-rally-up-to-43/slideshow/122796421.cms', published=datetime.datetime(2025, 7, 20, 15, 58, 41, tzinfo=tzoffset(None, 19800)), content=''),
 EconomicTime(title='How to Calculate Dividend Yield: A key metric for income investors', link='https://economictimes.indiatimes.com/markets/stocks/news/how-to-calculate-dividend-yield-a-key-metric-for-

In [32]:
#get the content by using beautiful soup
from bs4 import BeautifulSoup
import requests

for news in economic_times:
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(news.link, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    article_div = soup.find("article")
    if article_div is None:
        continue
    article_text = article_div.get_text(strip=True)
    news.content = article_text

In [33]:
#create LLM
groq_key = userdata.get('groq_key')
llm = ChatGroq(api_key=groq_key, model="qwen/qwen3-32b",max_retries=5)
llm_reasoning = ChatGroq(api_key=groq_key, model="meta-llama/llama-4-scout-17b-16e-instruct",max_retries=5)
llm_deep_seek = ChatGroq(api_key=groq_key, model="deepseek-r1-distill-llama-70b",max_retries=5)
search_tool = ExaSearchResults(exa_api_key= userdata.get('exa_api_key'))


In [34]:
class Stock(BaseModel):
    name: str = Field(description="Name of the stock")
    news_abstract: str = Field(description="Abstract of the news article related to the stock")

In [35]:
llm_stock_news = llm.with_structured_output(Stock)

In [36]:
stock_news:list[Stock] = []
def abstract_stock_new(news):
  prompt_stock = PromptTemplate.from_template("Following is the news:{news}. provide the company name and abstract news ").format(news=news.content)
  return llm_stock_news.invoke(prompt_stock)

for news in economic_times[:1]:
  stock_news.append(abstract_stock_new(news))


In [37]:
#Create State
class State(BaseModel):
    company_name: str = Field(description="Name of the company")
    news_abstract: str = Field(description="Abstract of the news article related to the company")
    queries:List[str] = []
    completed_queries: Annotated[List[str], operator.add] = []
    final_result: str = ''

class Query(BaseModel):
   query:str = ''

class Queries(BaseModel):
  queries:List[str] = []

In [38]:
#Worker State
class WorkerState(BaseModel):
    queries: str = ''
    completed_queries: Annotated[List[str], operator.add] = []

In [39]:
llm_queries = llm_reasoning.with_structured_output(Queries)
def orchestrator(state:State):
  promot_template = PromptTemplate.from_template("Your are Stock Market expert. Based on the following news list out the impacts in short / long:{news}").format(news=state.news_abstract)
  res = llm_queries.invoke(promot_template)
  return {'queries':res.queries}

def llm_call(query):
    """Worker queries web for queries"""
    search_results = search_tool._run(
          query=query['query'],
          num_results=1,
          text_contents_options=True,
          highlights=True,
      )
    prompt_stock = PromptTemplate.from_template("Following is the news:{news}. provide the abstract").format(news=search_results)
    res_abs = llm.invoke(prompt_stock)
    return {'completed_queries':[remove_think_blocks(res_abs.content)]} # Wrap the content in a list

def assign_workers(state: State):
    return [Send("llm_call", {"query": s}) for s in state.queries]

def synthesizer(state: State):
    body = "".join(state.completed_queries)
    promot_template = PromptTemplate.from_template("Your are Stock Market expert.Provide an abstract on the following news and do a analysis:{news}").format(news=body)
    res = llm_deep_seek.invoke(promot_template)
    return {"final_result": remove_think_blocks(res.content)}

def remove_think_blocks(text: str) -> str:
    # This pattern removes everything between <think> and </think>, including the tags
    return re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL).strip()


In [40]:
graph = StateGraph(State)
graph.add_node("orchestration", orchestrator)
graph.add_node("llm_call", llm_call)
graph.add_node("synthesizer", synthesizer)

graph.add_edge(START, "orchestration")
graph.add_conditional_edges("orchestration", assign_workers, ["llm_call"])
graph.add_edge("llm_call", "synthesizer")
graph.add_edge("synthesizer", END)

builder = graph.compile()

In [41]:
#state = State(company_name="TCS",news_abstract=)
selector = stock_news[0]
state = State(company_name=selector.name,news_abstract=selector.news_abstract)

In [42]:
res_date = builder.invoke(state)


In [43]:
res_date

{'company_name': 'ICICI Prudential Active Momentum Fund',
 'news_abstract': 'ICICI Prudential AMC has launched the Active Momentum Fund, focusing on earnings momentum rather than stock price trends. Managed by Manasvi Shah, the strategy prioritizes sustainable earnings growth to drive intrinsic value, aiming to avoid sentiment-driven market volatility. The fund employs a cycle-agnostic approach, blending bottom-up and top-down methods to diversify across sectors and market caps. It targets long-term alpha by leveraging earnings revisions, maintaining flexibility to adapt to macro events, and controlling turnover through stability in earnings-focused selections. The benchmark is the Nifty 500, reflecting its capitalization-agnostic stance.',
 'queries': ['ICICI Prudential Active Momentum Fund short-term impact',
  'ICICI Prudential Active Momentum Fund long-term impact'],
 'completed_queries': ['**Abstract:**  \nICICI Prudential AMC’s newly launched **Active Momentum Fund** redefines tr